In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2021/train.csv
/kaggle/input/tabular-playground-series-feb-2021/test.csv


In [2]:
#Importing necessary libraries

import torch
import torch.nn as nn
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

In [3]:
#Loading data

train_df = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')

In [4]:
train_df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [5]:
train_df.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'target'],
      dtype='object')

In [6]:
y = train_df['target']

In [7]:
#Seperating categorical and numerical features

categorical = [i for i in train_df.columns if 'cat' in i]
numerical = [i for i in train_df.columns if 'cont' in i]

In [8]:
ss = QuantileTransformer()
one_hot = ce.OneHotEncoder()

one_hot.fit(train_df[categorical])
ss.fit(train_df[numerical])

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


QuantileTransformer()

In [9]:
def preprocess(fr, train=True):
    if train:
        cat = fr[categorical]
        cat = one_hot.transform(cat)
        num = fr[numerical]
        num = pd.DataFrame(ss.transform(num), columns=numerical)
        cat_num = cat.merge(num, left_index=True, right_index=True)
        return cat_num
    
    else:
        cat = fr[categorical]
        cat = one_hot.transform(cat)
        num = fr[numerical]
        num = pd.DataFrame(ss.transform(num), columns=numerical)
        cat_num = cat.merge(num, left_index=True, right_index=True)
        return cat_num

In [10]:
len(preprocess(train_df).columns), len(preprocess(test_df, False).columns)

(70, 70)

In [11]:
inp = 70
hid = 440
hid2 = 530
hid3 = 378
hid4 = 137
hid5 = 52

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1    = nn.Linear(input_dim, num_hidden)
        self.linear2    = nn.Linear(num_hidden, hid2)
        self.linear3    = nn.Linear(hid2,hid3)
        self.linear4    = nn.Linear(hid3,hid4)
        self.linear5    = nn.Linear(hid4,hid5)
        self.batchNorm1 = nn.BatchNorm1d(num_hidden)
        self.batchNorm2 = nn.BatchNorm1d(hid2)
        self.batchNorm3 = nn.BatchNorm1d(hid3)
        self.batchNorm4 = nn.BatchNorm1d(hid4)
        self.batchNorm5 = nn.BatchNorm1d(hid5)
        self.dropout    = nn.Dropout(p=0.48)
        self.relu       = nn.ReLU()
        self.sigmoid    = nn.Sigmoid()
        self.out        = nn.Linear(hid5, 1)

    def forward(self, x):
        l1   = self.linear1(x)
        n1   = self.batchNorm1(l1)
        d1   = self.dropout(n1)
        act1 = self.relu(d1)
        l2   = self.linear2(act1)
        n2   = self.batchNorm2(l2)
        d2   = self.dropout(n2)
        act2 = self.relu(d2)
        l3   = self.linear3(act2)
        n3   = self.batchNorm3(l3)
        d3   = self.dropout(n3)
        act3 = self.relu(d3)
        l4   = self.linear4(act3)
        n4   = self.batchNorm4(l4)
        d4   = self.dropout(n4)
        act4 = self.relu(d4)
        l5   = self.linear5(act4)
        n5   = self.batchNorm5(l5)
        d5   = self.dropout(n5)
        act5 = self.relu(d5)
        l6   = self.out(act5)
        output = l6 #self.sigmoid(l6)
        return output

In [13]:
class RMSELoss(torch.nn.Module):
    def init(self):
        super(RMSELoss,self).init()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [14]:
loss = RMSELoss()
model = NeuralNetwork(inp,hid)

In [15]:
def torch_fit(x, y, x_val, y_val,model, loss, lr, num_epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,nesterov=False) 
    count = -1
    pvl = 0
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        y_pred_tensor = model(x)
        loss_value = loss(y_pred_tensor, y)
        val_loss   = loss(model(x_val),y_val)
        print(f'Epoch {epoch}, loss {loss_value.item():.4f} val loss: {val_loss.item():.4f} count: {count}')
        loss_value.backward()
        optimizer.step()
        if val_loss.item() > pvl:
            count += 1
        pvl = val_loss
        if count > 3750: 
            break
    return model

In [16]:
x = preprocess(train_df).values
x_train, x_val, y_train, y_val = train_test_split(x,y)
l = len(y_train)
lv = len(y_val)
l,lv

(225000, 75000)

In [17]:
x_tensor = torch.tensor(x_train).float().cuda()
x_val_tensor = torch.tensor(x_val).float().cuda()
y_tensor = torch.tensor(y_train.values).float().cuda()
y_val_tensor = torch.tensor(y_val.values).float().cuda()
y_tensor = y_tensor.view(l,1).cuda()
y_val_tensor = y_val_tensor.view(lv,1).cuda()

In [18]:
x_tensor.size(), y_tensor.size()

(torch.Size([225000, 70]), torch.Size([225000, 1]))

In [19]:
model.cuda()
nn_model = torch_fit(x_tensor, y_tensor, x_val_tensor, y_val_tensor, model=model, loss=loss, lr=0.00728, num_epochs=9550)

Epoch 0, loss 7.7290 val loss: 7.7312 count: -1
Epoch 1, loss 7.6628 val loss: 7.6682 count: 0
Epoch 2, loss 7.5395 val loss: 7.5411 count: 0
Epoch 3, loss 7.3626 val loss: 7.3684 count: 0
Epoch 4, loss 7.1394 val loss: 7.1471 count: 0
Epoch 5, loss 6.8775 val loss: 6.8831 count: 0
Epoch 6, loss 6.5783 val loss: 6.5869 count: 0
Epoch 7, loss 6.2508 val loss: 6.2559 count: 0
Epoch 8, loss 5.8967 val loss: 5.9024 count: 0
Epoch 9, loss 5.5219 val loss: 5.5280 count: 0
Epoch 10, loss 5.1331 val loss: 5.1399 count: 0
Epoch 11, loss 4.7364 val loss: 4.7415 count: 0
Epoch 12, loss 4.3359 val loss: 4.3369 count: 0
Epoch 13, loss 3.9367 val loss: 3.9400 count: 0
Epoch 14, loss 3.5590 val loss: 3.5620 count: 0
Epoch 15, loss 3.1968 val loss: 3.2009 count: 0
Epoch 16, loss 2.8843 val loss: 2.8897 count: 0
Epoch 17, loss 2.6318 val loss: 2.6303 count: 0
Epoch 18, loss 2.4521 val loss: 2.4500 count: 0
Epoch 19, loss 2.3513 val loss: 2.3449 count: 0
Epoch 20, loss 2.3264 val loss: 2.3274 count: 0
E

In [20]:
sample_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')
sample_df.head()

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5


In [21]:
tes_tensor = torch.tensor(preprocess(test_df, False).values).float().cuda()
tes_tensor.size()

torch.Size([200000, 70])

In [22]:
preds = model(tes_tensor)
preds = preds.detach().cpu().numpy()

In [23]:
sample_df.target = preds 
sample_df.head()

,id,target
0,0,7.635271
1,5,7.678309
2,15,7.592106
3,16,7.522267
4,17,7.270338


In [24]:
sample_df.to_csv('/kaggle/working/submission.csv', index=False)